# DATA PROCESSOR

This Jupiter notebook is used to clean fairly dirty dataset obtained by scraping the webpage of the Czech Chamber of Stomatology, merge it with data from the Czech Statistical Office and aggregate it to a more general geographical areas that will be further analysed. Output from this notebook are two CSV files with clean data on brick and regional level, respectively.

## Import of packages

In the entire notebook, only pandas will be applied:

In [1]:
import pandas as pd

## Loading data and initial adjustments

In [2]:
# loading data
data = pd.read_csv('dentists_wide.csv', index_col = 0)
# examining its structure
print(data.columns.values)

['name' 'wkp_name_1' 'wkp_name_2' 'wkp_name_3' 'wkp_name_4' 'wkp_name_5'
 'wkp_name_6' 'wkp_name_7' 'wkp_name_8' 'wkp_name_9' 'wkp_street_1'
 'wkp_street_2' 'wkp_street_3' 'wkp_street_4' 'wkp_street_5'
 'wkp_street_6' 'wkp_street_7' 'wkp_street_8' 'wkp_street_9' 'wkp_psc_1'
 'wkp_psc_2' 'wkp_psc_3' 'wkp_psc_4' 'wkp_psc_5' 'wkp_psc_6' 'wkp_psc_7'
 'wkp_psc_8' 'wkp_psc_9' 'wkp_town_1' 'wkp_town_2' 'wkp_town_3'
 'wkp_town_4' 'wkp_town_5' 'wkp_town_6' 'wkp_town_7' 'wkp_town_8'
 'wkp_town_9' 'wkp_phone_1' 'wkp_phone_2' 'wkp_phone_3' 'wkp_phone_4'
 'wkp_phone_5' 'wkp_phone_6' 'wkp_phone_7' 'wkp_phone_8' 'wkp_phone_9']


We scraped all available information about offices of each dentist. However, our goal is to analyse geographical distribution of the stomatological care. Therefore, for the purpose of our analysis, only names of dentists (to our best knowledge, the Czech Chamber of stomatology ensured it's uniqueness by changing title if neccessary) and the town his or her workplaces are located. Hence, we exlude interesting but for our analysis irrelevant information from our dataset.

In [3]:
# subseting dataset
columns = data.columns.to_series().str.contains('town')
columns[0] = True

data_sub = data[data.columns[columns]]
print(data_sub.columns.values)

['name' 'wkp_town_1' 'wkp_town_2' 'wkp_town_3' 'wkp_town_4' 'wkp_town_5'
 'wkp_town_6' 'wkp_town_7' 'wkp_town_8' 'wkp_town_9']


## Data cleaning

Having data in wide format is not very practical for our analysis:

We know that quite a few number of dentists are inactive (most of them retired). Those have NaN in all 9 slots for workplaces. 

Also, we have only 1 dentist that have 9 workplaces, therefore only 1 observation actually have full rank. 

In order to deal with this issue, long format is much more desirable:

In [4]:
data_long = pd.melt(data_sub, id_vars = 'name', 
                    value_vars = data_sub.columns.values[1:], 
                    value_name= 'Town')
print(data_long)

                                 name    variable                   Town
0                     MUDr. Amer Abed  wkp_town_1                  Plzeň
1            DDS. Ahmad Amin Abosaleh  wkp_town_1     Velký Třebešov 118
2            MUDr. Fatemeh Aboutorabi  wkp_town_1      Praha 5 - Smíchov
3              MUDr. Marie Abrahamová  wkp_town_1                  Plzeň
4               MUDr. Ahmad Abu Baker  wkp_town_1                Liberec
5                MUDr. František Adam  wkp_town_1                    NaN
6                      MUDr. Jan Adam  wkp_town_1     Jablonec nad Nisou
7                      MDDr. Jan Adam  wkp_town_1                   Brno
8               MUDr. Miluše Adamcová  wkp_town_1         Mladá Boleslav
9               MUDr. Libuše Adamcová  wkp_town_1                    NaN
10               MUDr. Alena Adamcová  wkp_town_1                 Přerov
11               MUDr. Silva Adamcová  wkp_town_1                    NaN
12             MUDr. Markéta Adamcová  wkp_town_1  

Even from this initial screening we see quite a few opportunities for cleaning: as mentioned already, majority of observations are NaNs, these will be dropped. 

Also, we notice that some "Towns" includes full adress by chaining "cislo popisne" with actual town, therefore we remove all numerical values from all strings.

This exclusion, however, generates many strings that have "whitespaces" - spaces before or after the actual string. We eliminate the whitespaces as well.

Last adjustment that is obviously needed is the standardisation of Prague's districts. We will be analysing Prague as a whole since, given the specifics of it's bricks, the analysis on lower level would not make sense. You cannot travel between bricks as easily as in Prague nowhere else in the Czech Republic (i.e. Prague 1 and Prague 2 vs. Cheb and Karlovy Vary).

In [5]:
# excluding NaN
data_long = data_long.dropna()
# excluding numbers from strings
data_long.loc[:,'Town'] = data_long.loc[:,'Town'].str.replace('\d+', '')
# getting rid of spaces before and after strings
data_long.loc[:,'Town'] = data_long.loc[:,'Town'].str.strip()
# getting rid of parts of Prague
data_long.loc[:,'Town'][data_long.loc[:,'Town'].str.contains('Praha')] = 'Praha'
print(data_long)

                                       name    variable                   Town
0                           MUDr. Amer Abed  wkp_town_1                  Plzeň
1                  DDS. Ahmad Amin Abosaleh  wkp_town_1         Velký Třebešov
2                  MUDr. Fatemeh Aboutorabi  wkp_town_1                  Praha
3                    MUDr. Marie Abrahamová  wkp_town_1                  Plzeň
4                     MUDr. Ahmad Abu Baker  wkp_town_1                Liberec
6                            MUDr. Jan Adam  wkp_town_1     Jablonec nad Nisou
7                            MDDr. Jan Adam  wkp_town_1                   Brno
8                     MUDr. Miluše Adamcová  wkp_town_1         Mladá Boleslav
10                     MUDr. Alena Adamcová  wkp_town_1                 Přerov
12                   MUDr. Markéta Adamcová  wkp_town_1                Polička
13                     MUDr. Ivana Adamcová  wkp_town_1  Jablonné v Podještědí
14                  MDDr. Adéla Adamczyková  wkp_tow

After the brief cleaning we will try to map the bricks and regions to our towns using bridge file downloaded from https://www.cuzk.cz

The bridge file have following structure:

In [6]:
bridge = pd.read_csv("TOWN_BRICK_REGION.csv")
print(bridge.head())

      NAZEV             OKRES                  KRAJ
0  Abertamy      Karlovy Vary      Karlovarský kraj
1    Adamov        Kutná Hora      Středočeský kraj
2    Adamov  České Budějovice        Jihočeský kraj
3    Adamov           Blansko     Jihomoravský kraj
4  Adršpach            Náchod  Královéhradecký kraj


In [7]:
#counting maximum duplicates of one town name
print(bridge['NAZEV'].value_counts().max())

14


Clearly, we have an issue here with small villages accross the Czech Republic regions that have the same name.

From the data on https://www.dent.cz/zubni-lekari we have no way of knowing from which 'Adamov' the dentist is (there is only information about town). Even if we tried to include whole adress, we argue that it would not help very much - vast majority of such villages have only one street, hence whole adress would be only for instance 'Adamov 1' - again, no way of distinguishing between the 3 Adamovs.

The approach we selected as "the best" is to include all such places and weight the dentists based on number of occurences in the dataset (i.e. if we have a dentist that is in the dataset n times his weight for all such observations would be 1/n).

Our believe that this is the best approach is based on two arguments:

1. It will minimise the impact of the duplicates since the sum of the dentists will be the actual number of active dentists and we will ensure that at least part of the dentist is on the right place (luckily, as shown bellow the number of duplicated dentists is not very high)
2. It will discriminate between dentists with one vs. multiple offices (clearly, if you are in one town all the time you have greater impact on the overall availability of stomatological care compared to someone who travels to several towns)

Merging the two datasets yield following dataset. Note that we have only around 600 new observations compared to the unmerged dataset. Given the fact that one out of nearly 10 000 original observation could be responsible for 14 such duplicates, than, given our weightening methodology, we argue that this issue will not have significant effect on our results.

**We also have first conclusion: luckily, dentists tend to concentrate in larger towns**

In [8]:
data_merged = pd.merge(data_long, bridge, how= 'left', left_on= 'Town', right_on = 'NAZEV')
print(data_merged)

                                   name    variable                   Town  \
0                       MUDr. Amer Abed  wkp_town_1                  Plzeň   
1              DDS. Ahmad Amin Abosaleh  wkp_town_1         Velký Třebešov   
2              MUDr. Fatemeh Aboutorabi  wkp_town_1                  Praha   
3                MUDr. Marie Abrahamová  wkp_town_1                  Plzeň   
4                 MUDr. Ahmad Abu Baker  wkp_town_1                Liberec   
5                        MUDr. Jan Adam  wkp_town_1     Jablonec nad Nisou   
6                        MDDr. Jan Adam  wkp_town_1                   Brno   
7                 MUDr. Miluše Adamcová  wkp_town_1         Mladá Boleslav   
8                  MUDr. Alena Adamcová  wkp_town_1                 Přerov   
9                MUDr. Markéta Adamcová  wkp_town_1                Polička   
10                 MUDr. Ivana Adamcová  wkp_town_1  Jablonné v Podještědí   
11              MDDr. Adéla Adamczyková  wkp_town_1             

As expected, further cleaning of the data from the Chamber of Stomatology was required as around 1 000 observations could not be originally matched.

The majority of them was caused by districts of larger towns other than Prague, these were resolved in the same fashion by a simple loop over a list of towns.

Other, approximatelly 200, had to be resolved manually as they were caused by misspelling, other notation and, actually majority of them, small vilages written in as: 'Village u Town' therefore, in order to avoid duplicates as much as possible these observations were assigned to "Town". 

The second cleaning was done using loop over two ordered lists - one with values to be replaced and one with values to be filled.

In [9]:
# deleting parts of the town
strings_1 = ['Pardubice',
            'Havířov',
            'Brno',
            'Ostrava',
            'Plzeň',
            'Karviná',
            'Litoměřice',
            'Třinec',
            'Klatovy',
            'Olomouc',
            'Hranice',
            'Kutná Hora',
            'Karlovy Vary',
            'Říčany',
            'Zruč',
            'Třemošná',
            'Bohumín',
            'Hradec Králové',
            'Jičín',
            'Domažlice',
            'Mariánské Údolí',
            'Opava',
            'Rájec',
            'Herálec',
            'Poděbrady',
            'Přerov',
            'Benešov',
            'Kolín',
            'Liberec',
            'Rokycany',
            'Chrudim',
            'Vodňany',
            'Orlová',
            'Jaroměř',
            'Strakonice',
            'Neratovice',
            'Hutisko',
            'Zlín',
            'Klatovy',
            'Zlatníky',
            'Straškov',
            'Uherské Hradiště',
            'Trutnov',
            'Jindřichův Hradec',
            'Valašské Meziříčí',
            'Litultovice',
            'Hvozd',
            'Valašská Polanka',
            'Krhanice',
            'Záhřeb',
            'Adršpach',
            'Vysoké Mýto',
            'Příbram']

for i in strings_1:
    data_long.loc[:,'Town'][data_long.loc[:,'Town'].str.contains(i)] = i


# other corrections
string_2_in =  ['Frýdek - Místek',
                'Frýdek- Místek',
                'Frýdek Místek',
                'Kolán',
                'Havl. Brod',
                'LIberec',
                'Plze',
                'Karlových Var',
                'Uherského Hradiště',
                'ŽDÍREC NAD DOUBRAVOU',
                'LItoměřice',
                'Brandýs nad Labem - Stará Boleslav',
                'Hlinsko v Čechách',
                'Zbýšov u Brna',
                'Mikulov na Moravě',
                'Lom u Mostu',
                'Bystřice nad Olší',
                'Planá u Mariánských Lázní',
                'Petrovice u Ústí nad Labem',
                'Hustopeče u Brna',
                'Chlumčany u Přeštic',
                'Jesenice u Rakovníka',
                'Šlapanice u Brna',
                'Ostrov nad Ohří',
                'Trstěnice u Litomyšle',
                'Roztoky u Prahy',
                'Zruč',
                'Nové Sedlo u Lokte',
                'Hořice v Podkrkonoší',
                'Bečov u Mostu',
                'Horní Těrlicko',
                'Šenov u Ostravy',
                'Meziboří u Litvínova',
                'Brandýs nad Labem',
                'Studenec u Horek',
                'Mikulovice u Znojma',
                'Chrást u Plzně',
                'Petřvald u Karviné',
                'Mariánské Údolí',
                'Bystré u Poličky',
                'Milovice nad Labem',
                'Lužná v Čechách',
                'Kostelec u Jihlavy',
                'Šumvald u Uničova',
                'Krásná Lípa u Rumburka',
                'Věkoše',
                'Studnice u Náchoda',
                'Habartice u Frýdlantu',
                'Jindřichov u Šumperka',
                'Zastávka u Brna',
                'Staré Město pod Sněžníkem',
                'Holice v Čechách',
                'Březová u Sokolova',
                'Bohdalice',
                'Bělá nad Radbůzou',
                'Jesenice u Prahy',
                'Újezd u Valašských Klobouk',
                'Dvůr Králové',
                'Loučka u Valašského Meziříčí',
                'Staré Město č.',
                'Zeleneč v Čechách',
                'Frýdlant v Čechách',
                'Krnov, Pod Cilínem',
                'Studná',
                'Králův Dvůr u Berouna',
                'Hať u Hlučína',
                'Petrovice u Karviné č.',
                'Meziměstí u Broumova',
                'Rudník u Vrchlabí',
                'Hutisko',
                'Štítná nad Vláří',
                'Kobylí na Moravě',
                'Opařany č.p.',
                'Sněžné na Moravě',
                'Klašterec nad Ohří',
                'Mokrá',
                'Zlatníky',
                'Straškov',
                'Teplá u Toužimě',
                'Košťany u Teplic',
                'Jílové u Děčína',
                'Morkovice',
                'Petrovice u Sedlčan',
                'Záhřeb',
                'Zábřeh na Moravě',
                'Rudná u Prahy',
                'Troubky nad Bečvou',
                'Vedryně',
                'Brozany',
                'Bor u Tachova',
                'Rychvald u Karviné',
                'Hošťálková u Vsetína',
                'Černožice nad Labem',
                'Ústí nad Labem-Klíše',
                'Dubí u Teplic',
                'Boršice u Buchlovic',
                'Podolí u Brna',
                'Kunžák',
                'Višňové u Znojma',
                'Harrachov v Krkonoších',
                'Kostelec nad Černými Lesy',
                'Javorník u Jeseníku',
                'Olešnice na Moravě',
                'Černovice u Tábora',
                'Čepí č.p.',
                'Proseč u Skutče',
                'Bernartice u Milevska',
                'Lipová - lázně',
                'Janovice č.',
                'Křenovice u Slavkova',
                'Lučina č.',
                'Oskava č.p.',
                'Rousínov u Vyškova',
                'Dolní Sloupnice',
                'Hovorany č.',
                'Jakubčovice',
                'Kamenice n.Lipou',
                'Jablůnka nad Bečvou',
                'Dolní Bečva č.',
                'Svatobořice Mistřín',
                'Miřetice u Hlinska',
                'Písek u Jablunkova',
                'Morkovice - Slížany',
                'LItvínov',
                'Kojetín I - Město',
                'Sedlec - Prčice',
                'Mýto v Čechách',
                'Březolupy č.p.',
                'Kamenice u Jihlavy',
                'Kralupy nad Vltavou-Lobeček',
                'Dolní Újezd u Lipníka nad Bečvou',
                'Lednice na Moravě',
                'Novosedly na Moravě',
                'Zvole u Zábřeha',
                'Vratislavice nad Nisou',
                'Rosice u Brna',
                'Lípa č.p.',
                'Sloup v Moravském Krasu',
                'Děčín  - Nové Město',
                'Ořechov u Brna',
                'Kokory č.',
                'Blatnice pod svatým Antonínkem',
                'Lomnice u Tišnova',
                'Sušice II',
                'Lužice u Hodonína',
                'Čistá u Rakovníka',
                'Hrádek nad Olší',
                'Nezamyslice u Prostějova',
                'Návsí u Jablunkova',
                'Opatov v Čechách',
                'Třebíč - Zámostí',
                'Stará Boleslav',
                'Hlinsko pod Hostýnem č.',
                'Studénka, Butovice',
                'Hrádek u Rokycan',
                'Radnice u Rokycan',
                'Vimperk II',
                'Moravany u Holic',
                'Beroun - Závodí',
                'Osek u Duchcova',
                'Kamenice - Olešovice',
                'Brandýs nad Labem -Stará Boleslav',
                'Chlumec nad Cidlinou IV',
                'Soběslav I',
                'Albrechtice u Českého Těšína',
                'Brumov - Bylnice',
                'Budišov u Třebíče',
                'Holubice - Kozinec',
                'Hostomice pod Brdy',
                'Stránčice',
                'Vraný u Slaného',
                'Chvalkovice v Čechách',
                'Temelín - elektrárna',
                'Vrbátky č.',
                'Brodce nad Jizerou',
                'Záhoří u Písku',
                'Rohozná u Poličky',
                'Borotín u Tábora',
                'Vsetín - Trávníky',
                'Stonava /',
                'Chvalšiny č.p.',
                'Soběšice u Sušic',
                'Činěves č.',
                'Smirady',
                'Luby u Chebu',
                'Dětmarovice č.',
                'Desná v Jizerských horách',
                'Drahotuše',
                'Stráž pod Rálskem',
                'Libina č.p.',
                'Všeruby u Plzně',
                'Hovězí č.p.',
                'Liběšice u Litoměřic',
                'Markvartice u Děčína',
                'Lukavec u Pacova',
                'Borová u Poličky',
                'Bratříkov',
                'Marianské Lázné',
                'Studnice u Vyškova',
                'Brandýs',
                'Hutisko',
                'Jakubčovice',
                'Vratislavice',
                'Boleslav',
                'Smirady',
                'Drahotuše',
                'Alšovice'
                ]

string_2_out = ['Frýdek-Místek',
                'Frýdek-Místek',
                'Frýdek-Místek',
                'Kolín',
                "Havlíčkův Brod",
                "Liberec",
                "Plzeň",
                'Karlovy Vary',
                'Uherské Hradiště',
                'Ždírec nad Doubravou',
                'Litoměřice',
                'Brandýs nad Labem-Stará Boleslav',
                "Hlinsko",
                "Brno",
                "Mikulov",
                "Most",
                "Bystřice",
                "Mariánské Lázně",
                "Ústí nad Labem",
                "Brno",
                "Přeštice",
                "Rakovník",
                "Brno",
                "Ostrov",
                "Litomyšl",
                "Roztoky",
                "Zruč-Senec",
                "Loket",
                "Hořice",
                "Most",
                "Těrlicko",
                "Ostrava",
                "Litvínov",
                "Brandýs",
                "Horky",
                "Znojmo",
                "Plzeň",
                "Karviná",
                "Litvínov",
                "Polička",
                "Nymburk",
                "Rakovník",
                "Jihlava",
                "Uničov",
                "Rumburk",
                "Hradec Králové",
                "Náchod",
                "Frýdlant",
                "Šumperk",
                "Brno",
                "Jindřichov",
                "Pardubice",
                "Sokolov",
                "Pavlovice",
                "Rozvadov",
                "Praha",
                "Valašské Klobouky",
                "Trutnov",
                "Valašské Meziříčí",
                "Staré Město",
                "Zeleneč",
                "Frýdlant",
                "Krnov",
                "Studná",
                "Beroun",
                "Hlučín",
                "Karviná",
                "Broumov",
                "Vrchlabí",
                "Hutisko",
                "Zlín",
                "Terezín",
                "Opařany",
                "Sněžné",
                "Klášterec nad Ohří",
                "Brno",
                "Zlatníky-Hodkovice",
                "Straškov-Vodochody",
                "Toužim",
                "Teplice",
                "Děčín",
                "Morkovice-Slížany",
                "Sedlčany",
                "Šumperk",
                "Šumperk",
                "Praha",
                "Troubky",
                "Vendryně",
                "Pardubice",
                "Tachov",
                "Karviná",
                "Vsetín",
                "Hradec Králové",
                "Ústí nad Labem",
                "Teplice",
                "Buchlovice",
                "Brno",
                "Jindřichův Hradec",
                "Znojmo",
                "Harrachov",
                "Jevany",
                "Jeseník",
                "Olešnice",
                "Tábor",
                "Čepí",
                "Proseč",
                "Bernartice",
                "Lipová",
                "Janovice",
                "Křenovice",
                "Lučina",
                "Oskava",
                "Rousínov",
                "Sloupnice",
                "Hovorany",
                "Jakubčovice",
                "Kamenice",
                "Jablůnka",
                "Dolní Bečva",
                "Svatobořice-Mistřín",
                "Miřetice",
                "Písek",
                "Morkovice-Slížany",
                "Litvínov",
                "Kojetín",
                "Sedlec-Prčice",
                "Mýto",
                "Březolupy",
                "Kamenice",
                "Kralupy nad Vltavou",
                "Újezd",
                "Lednice",
                "Novosedly",
                "Zvole",
                "Vratislavice",
                "Rosice",
                "Lípa",
                "Sloup",
                "Děčín",
                "Ořechov",
                "Kokory",
                "Blatnice",
                "Lomnice",
                "Sušice",
                "Lužice",
                "Rakovník",
                "Hrádek",
                "Nezamyslice",
                "Návsí",
                "Opatov",
                "Třebíč",
                "Boleslav",
                "Hlinsko",
                "Studénka",
                "Rokycany",
                "Rokycany",
                "Vimperk",
                "Moravany",
                "Beroun",
                "Osek",
                "Kamenice",
                "Brandýs nad Labem",
                "Chlumec nad Cidlinou",
                "Soběslav",
                "Albrechtice",
                "Brumov",
                "Budišov",
                "Holubice",
                "Hostomice",
                "Říčany",
                "Vraný",
                "Chvalkovice",
                "Temelín",
                "Vrbátky",
                "Mladá Boleslav",
                "Záhoří",
                "Rohozná",
                "Borotín",
                "Vsetín",
                "Stonava",
                "Chvalšiny",
                "Soběšice",
                "Činěves",
                "Smirady",
                "Luby",
                "Dětmarovice",
                "Desná",
                "Drahotuše",
                "Stráž",
                "Libina",
                "Všeruby",
                "Hovězí",
                "Liběšice",
                "Markvartice",
                "Lukavec",
                "Borová",
                "Alšovice",
                "Mariánské Lázně",
                "Studnice",
                "Brandýs nad Labem-Stará Boleslav",
                "Rožnov pod Radhoštěm",
                "Jakubčovice nad Odrou",
                "Liberec",
                "Mladá Boleslav",
                "Smidary",
                "Lipník nad Bečvou",
                "Pěnčín"
                ]

for i in range(len(string_2_in)):
    data_long.loc[:,'Town'][data_long.loc[:,'Town'].str.contains(string_2_in[i])] = string_2_out[i]


data_merged = pd.merge(data_long, bridge, how= 'left', left_on= 'Town', right_on = 'NAZEV')

In [10]:
missing = data_merged[data_merged['NAZEV'].isna()]['Town'].unique()
print(len(missing))

1


In [11]:
print(missing)

['Studná']


It is not possible to find "Studna" even on google maps, hence this one observation will be dropped

In [12]:
data_merged = data_merged.dropna()

With the data cleaned and merged the last step before aggregation is the creation of "Weighted dentist" as described above.

Note that, as already mentioned, Chamber of Stomatology ensured that variable names uniquely identifies a dentist, hence the reasons for multiple occurences are:

1. Multiple offices
2. Office in a small town whose name is not unique and there was no additional info on the pages of the Chamber of Stomatology

In [13]:
data_merged['Dentist_count'] = data_merged.groupby('name')['name'].transform('size')
data_merged['Dentist_weight'] = 1/data_merged['Dentist_count']
print(data_merged)

                                   name    variable                   Town  \
0                       MUDr. Amer Abed  wkp_town_1                  Plzeň   
1              DDS. Ahmad Amin Abosaleh  wkp_town_1         Velký Třebešov   
2              MUDr. Fatemeh Aboutorabi  wkp_town_1                  Praha   
3                MUDr. Marie Abrahamová  wkp_town_1                  Plzeň   
4                 MUDr. Ahmad Abu Baker  wkp_town_1                Liberec   
5                        MUDr. Jan Adam  wkp_town_1     Jablonec nad Nisou   
6                        MDDr. Jan Adam  wkp_town_1                   Brno   
7                 MUDr. Miluše Adamcová  wkp_town_1         Mladá Boleslav   
8                  MUDr. Alena Adamcová  wkp_town_1                 Přerov   
9                MUDr. Markéta Adamcová  wkp_town_1                Polička   
10                 MUDr. Ivana Adamcová  wkp_town_1  Jablonné v Podještědí   
11              MDDr. Adéla Adamczyková  wkp_town_1             

## Data agregation and final adjustments

Now we are ready to aggregate the data on brick level:

In [14]:
dentists_brick = data_merged.groupby('OKRES', as_index = False).agg({"Dentist_weight":"sum"})

dentists_brick = dentists_brick.rename(columns={'Dentist_weight': 'DENTISTS'})

print(dentists_brick.head())

         OKRES    DENTISTS
0      Benešov   59.294048
1       Beroun   57.342857
2      Blansko   87.608333
3   Brno-město  613.641667
4  Brno-venkov   81.759524


Having only information about the number of dentists per brick is not sufficent for the evaluation of the stomatological care availability. We need the data about relevant population as well. 

We obtained relatively recent data (31.12.2018) about population on brick level from the Czech Statistical Office open database (https://vdb.czso.cz/)

In [15]:
population_brick = pd.read_csv('populace_okresy.csv')

print(population_brick.head())

        OKRES  POPULACE
0     Benešov     98708
1      Beroun     93726
2      Kladno    165271
3       Kolín    101604
4  Kutná Hora     75370


Before merging these two dataframes we check whether they are compatible - only one small adjustment needed:

In [16]:
print(set(dentists_brick['OKRES']) - set(population_brick['OKRES']))

{'Hlavní město Praha'}


In [17]:
population_brick.at[9, 'OKRES'] = 'Hlavní město Praha'
print(set(dentists_brick['OKRES']) - set(population_brick['OKRES']))

set()


After merging the data, we are ready to analyse stomatological care availability per brick.

In [18]:
brick_data_final = pd.merge(dentists_brick, population_brick, how= "left", on= 'OKRES')

brick_data_final['PATIENTS_PER_DENTIST'] = brick_data_final['POPULACE']/brick_data_final['DENTISTS']

print(brick_data_final)

                  OKRES     DENTISTS  POPULACE  PATIENTS_PER_DENTIST
0               Benešov    59.294048     98708           1664.720220
1                Beroun    57.342857     93726           1634.484305
2               Blansko    87.608333    108801           1241.902407
3            Brno-město   613.641667    380681            620.363676
4           Brno-venkov    81.759524    222370           2719.805469
5               Bruntál    46.266667     92018           1988.861671
6               Břeclav    66.066667    115906           1754.379415
7                  Cheb    62.316667     91540           1468.948917
8              Chomutov    59.000000    124751           2114.423729
9               Chrudim    92.992857    104344           1122.064675
10            Domažlice    37.866667     61902           1634.735915
11                Děčín    58.433333    129831           2221.865374
12        Frýdek-Místek   147.745238    214187           1449.704930
13       Havlíčkův Brod    67.4428

We will devote special Jupyter notebook for visualisation and analyses, right now we only export our cleaned and ready to use dataset

In [19]:
brick_data_final.to_csv("brick_data_final.csv")

We will also look at the more general geographical units - Czech regions. In order to get the needed bridge, we will juts slightly adjust already introduced original bridge (excluding towns and duplicates)

In [20]:
bridge_2 = bridge[['OKRES','KRAJ']].drop_duplicates()
print(bridge_2)

                    OKRES                  KRAJ
0            Karlovy Vary      Karlovarský kraj
1              Kutná Hora      Středočeský kraj
2        České Budějovice        Jihočeský kraj
3                 Blansko     Jihomoravský kraj
4                  Náchod  Královéhradecký kraj
5         Ústí nad Orlicí       Pardubický kraj
6                 Karviná  Moravskoslezský kraj
7     Rychnov nad Kněžnou  Královéhradecký kraj
8                   Písek        Jihočeský kraj
9      Jablonec nad Nisou        Liberecký kraj
10             Nový Jičín  Moravskoslezský kraj
11              Prostějov        Olomoucký kraj
12                Bruntál  Moravskoslezský kraj
15                Hodonín     Jihomoravský kraj
16              Pelhřimov         Kraj Vysočina
17                Jihlava         Kraj Vysočina
18                  Děčín          Ústecký kraj
19                   Cheb      Karlovarský kraj
20       Uherské Hradiště          Zlínský kraj
21           Praha-východ      Středočes

Mapping regions to bridge and further aggregating we easily get to the desired dataset at more general level:

In [21]:
regional_data_final = pd.merge(brick_data_final, bridge_2, how= "left", on= 'OKRES')

regional_data_final = regional_data_final.groupby('KRAJ', as_index = False).agg({"DENTISTS":"sum", "POPULACE":"sum"})

regional_data_final['PATIENTS_PER_DENTIST'] = regional_data_final['POPULACE']/regional_data_final['DENTISTS']

print(regional_data_final)

                    KRAJ     DENTISTS  POPULACE  PATIENTS_PER_DENTIST
0     Hlavní město Praha  1825.233333   1308632            716.966963
1      Jihomoravský kraj  1036.176190   1187667           1146.201786
2         Jihočeský kraj   428.869048    642133           1497.270562
3       Karlovarský kraj   184.692857    294896           1596.683297
4          Kraj Vysočina   364.566667    509274           1396.929688
5   Královéhradecký kraj   477.078571    551021           1154.990044
6         Liberecký kraj   293.226190    442356           1508.582843
7   Moravskoslezský kraj   805.445238   1203299           1493.955074
8         Olomoucký kraj   569.169048    632492           1111.255088
9        Pardubický kraj   355.711905    520316           1462.745534
10         Plzeňský kraj   504.802381    584672           1158.219577
11      Středočeský kraj   634.965476   1369332           2156.545594
12          Zlínský kraj   432.860714    582921           1346.671067
13          Ústecký 

In [22]:
regional_data_final.to_csv("regional_data_final.csv")